In [172]:
import torch

import torchvision
from torch.utils.data import DataLoader , TensorDataset
import torch.nn as nn

import torch.nn.functional as F

import matplotlib.pyplot as plt

import pandas as pd

import numpy as np

from torch.utils.data.dataset import random_split


from tqdm import tqdm

In [173]:
sales = pd.read_csv('Social_Network_Ads.csv')
sales.head()

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0


In [174]:
df = sales.copy(deep=True)
df.head()

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0


In [175]:
df  = df[['Gender' , 'Age' , 'EstimatedSalary' , 'Purchased']]

In [176]:
df

,Gender,Age,EstimatedSalary,Purchased
0,Male,19,19000,0
1,Male,35,20000,0
2,Female,26,43000,0
3,Female,27,57000,0
4,Male,19,76000,0
...,...,...,...,...
395,Female,46,41000,1
396,Male,51,23000,1
397,Female,50,20000,1
398,Male,36,33000,0


In [177]:
df['Male'] = pd.get_dummies(df['Gender'] , drop_first=True)

In [178]:
df  = df.drop('Gender' , 1)

In [179]:
df

,Age,EstimatedSalary,Purchased,Male
0,19,19000,0,1
1,35,20000,0,1
2,26,43000,0,0
3,27,57000,0,0
4,19,76000,0,1
...,...,...,...,...
395,46,41000,1,0
396,51,23000,1,1
397,50,20000,1,0
398,36,33000,0,1


## Now change to numpy 


In [239]:
numpy_input = df[df.columns[[0,1,3]]].values
numpy_target = df[df.columns[[2]]].values

In [252]:
numpy_input.size , numpy_target.size

(1200, 400)

## Change numpy to tensor

In [253]:
tensor_input = torch.from_numpy(numpy_input).type(torch.float32)
tensor_target = torch.from_numpy(numpy_target).type(torch.float32)

In [254]:
tensor_input.size()

torch.Size([400, 3])

In [255]:
tensor_target.size()

torch.Size([400, 1])

In [226]:
dataset = TensorDataset(tensor_input , tensor_target)

## Split and Create a DataLoader

In [256]:
train_ds , val_ds = random_split(dataset , [300,100])

In [257]:
batch_size=50

train_loader= DataLoader(train_ds , batch_size=batch_size , shuffle=True)
val_loader = DataLoader(val_ds ,batch_size=batch_size)

In [268]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3,64)
        self.linear2 = nn.Linear(64,32)
        self.linear3 = nn.Linear(32,1)
        
    def forward(self,x):
        x = x.view(-1,3)
        x = F.relu(self.linear(x))
        x = F.relu(self.linear2(x))
        x = F.relu(self.linear3(x))
        return x
    

model = Net()

criterion = nn.BCEWithLogitsLoss()


optimizer = torch.optim.Adam(model.parameters() , lr=0.001)



epochs = 10

for epoch in tqdm(range(epochs)):
    for data,target in train_loader:
        
        output = model(data)
        
        #loss
        
        loss = criterion(output ,target.reshape(50,1))
        
        #backpropagation
        
        model.zero_grad()
        
        loss.backward()
        
        optimizer.step()
    print(f'Loss : {loss}')
        

  0%|                                                                                           | 0/10 [00:00<?, ?it/s]

Loss : 0.6931471824645996
Loss : 0.6931471824645996
Loss : 0.6931471824645996
Loss : 0.6931471824645996
Loss : 0.6931471824645996
Loss : 0.6931471824645996


 60%|█████████████████████████████████████████████████▊                                 | 6/10 [00:00<00:00, 56.08it/s]

Loss : 0.6931471824645996
Loss : 0.6931471824645996
Loss : 0.6931471824645996
Loss : 0.6931471824645996


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 55.56it/s]


In [279]:
with torch.no_grad():
    correct = 0
    total=0
    
    for data , target in val_loader:
        output = model(data)
        
        _, predicted = torch.max(output.data, 1)
                
        total += target.size(0)
        correct += (predicted == target).sum().item()

    print('Test Accuracy of the model on the 100 test is: {} %'.format(100 * correct / total))
            

Test Accuracy of the model on the 100 test is: 64.0 %


In [280]:
# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')